<a href="https://www.kaggle.com/code/davidarciniegas16/proyecto-ri-vinos-arciniegas-quirola?scriptVersionId=285233700" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wine-reviews/winemag-data_first150k.csv
/kaggle/input/wine-reviews/winemag-data-130k-v2.json
/kaggle/input/wine-reviews/winemag-data-130k-v2.csv


In [2]:
df = pd.read_csv('/kaggle/input/wine-reviews/winemag-data-130k-v2.csv')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


# Dependencias

In [3]:
import pandas as pd
import numpy as np
import string
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from collections import defaultdict
import math
from collections import Counter

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Configuración global
STOPWORDS = set(stopwords.words('english'))
STEMMER = SnowballStemmer('english')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# 1. Preprocesamiento y Construcción del Corpus ###
### Funcion para normalizar el texto realizar tokenización, minúsculas, eliminación de stopwords y stemming.

In [4]:
def preprocesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    # Tokenización
    tokens = word_tokenize(texto)
    # Stopwords y Stemming 
    tokens_limpios = [
        STEMMER.stem(token) 
        for token in tokens 
        if token not in STOPWORDS and token.isalpha()
    ]
    return tokens_limpios

def calcular_idf(corpus_tokens):
    N = len(corpus_tokens)
    conteo_docs = Counter()
    for tokens in corpus_tokens:
        conteo_docs.update(set(tokens))
    
    idf_dict = {
        palabra: math.log10(N / (freq + 1)) 
        for palabra, freq in conteo_docs.items()
    }
    return idf_dict

def generar_vector_tfidf(tokens, idf_dict):
    tf = Counter(tokens)
    total_tokens = len(tokens)
    vector = {}
    
    for palabra, conteo in tf.items():
        if palabra in idf_dict:
            # TF = frecuencia / total
            tf_val = conteo / total_tokens
            # TF-IDF = TF * IDF
            vector[palabra] = tf_val * idf_dict[palabra]
    return vector

def similitud_coseno(vec1, vec2):
    interseccion = set(vec1.keys()) & set(vec2.keys())
    
    numerador = sum([vec1[x] * vec2[x] for x in interseccion])
    
    suma1 = sum([vec1[x]**2 for x in vec1.keys()])
    suma2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominador = math.sqrt(suma1) * math.sqrt(suma2)
    
    if not denominador:
        return 0.0
    return numerador / denominador


df = df.dropna(subset=['description']).reset_index(drop=True)

print(f"Procesando {len(df)} documentos...")

# Aplicar preprocesamiento
df['tokens'] = df['description'].apply(preprocesar_texto)
df['texto_procesado'] = df['tokens'].apply(lambda x: ' '.join(x))

print("Preprocesamiento completado.\n")

Procesando 129971 documentos...
Preprocesamiento completado.



In [5]:
# Ejemplo de Preprocesamiento
texto_ejemplo = "The Red Wine has a Fruity aroma!!, delicious tropical warm"
tokens_ejemplo = preprocesar_texto(texto_ejemplo)
print(f"1. Texto Original: '{texto_ejemplo}'")
print(f"   Tokens Limpios: {tokens_ejemplo}")

# Ejemplo de IDF
idf_demo = calcular_idf(df['tokens'])
# Mostramos el valor IDF de una palabra común 
ej_comun = tokens_ejemplo[0] if tokens_ejemplo else "wine"
print(f"\n2. Valor IDF:")
print(f"   Palabra '{ej_comun}': {idf_demo.get(ej_comun, 0):.4f} (Menor valor = más común)")

# Ejemplo de Vector TF-IDF
vector_ejemplo = generar_vector_tfidf(tokens_ejemplo, idf_demo)
print(f"\n3. Vector Generado (Diccionario de Pesos):")
print(f"   {vector_ejemplo}")

1. Texto Original: 'The Red Wine has a Fruity aroma!!, delicious tropical warm'
   Tokens Limpios: ['red', 'wine', 'fruiti', 'aroma', 'delici', 'tropic', 'warm']

2. Valor IDF:
   Palabra 'red': 0.8817 (Menor valor = más común)

3. Vector Generado (Diccionario de Pesos):
   {'red': 0.1259519751764652, 'wine': 0.042078008752810404, 'fruiti': 0.15105155249591645, 'aroma': 0.07239987352491876, 'delici': 0.1905659281756981, 'tropic': 0.2216845186808789, 'warm': 0.2396691161973964}


# 2. Indexación (Índice Invertido) ###

In [6]:
def construir_indice_invertido(lista_tokens):
    indice_invertido = defaultdict(list)
    
    for id_doc, tokens in enumerate(lista_tokens):
        # Contar frecuencia en el documento actual
        frecuencia_termino = defaultdict(int)
        for token in tokens:
            frecuencia_termino[token] += 1
            
        # Agregar al índice invertido
        for token, freq in frecuencia_termino.items():
            indice_invertido[token].append((id_doc, freq))
            
    return indice_invertido

# Construir índice
indice_invertido = construir_indice_invertido(df['tokens'])
print(f"Índice construido con {len(indice_invertido)} términos únicos.\n")


print("Muestra del Índice Invertido:")
print("-" * 50)
for i, (termino, posteos) in enumerate(indice_invertido.items()):
    if i >= 5: break  # cantidad de termino a mostrar
    print(f"Término: '{termino}'")
    print(f"  Aparece en {len(posteos)} documentos.")
    print(f"  Posteos (ID_Doc, Frec): {posteos[:3]} ...") 
    

diccionario_idf = calcular_idf(df['tokens'])

# Generar vectores TF-IDF para cada documento 
df['vector_tfidf'] = df['tokens'].apply(lambda x: generar_vector_tfidf(x, diccionario_idf))



Índice construido con 30929 términos únicos.

Muestra del Índice Invertido:
--------------------------------------------------
Término: 'aroma'
  Aparece en 40461 documentos.
  Posteos (ID_Doc, Frec): [(0, 1), (3, 1), (5, 1)] ...
Término: 'includ'
  Aparece en 1757 documentos.
  Posteos (ID_Doc, Frec): [(0, 1), (13, 1), (34, 1)] ...
Término: 'tropic'
  Aparece en 3647 documentos.
  Posteos (ID_Doc, Frec): [(0, 1), (34, 1), (43, 1)] ...
Término: 'fruit'
  Aparece en 51820 documentos.
  Posteos (ID_Doc, Frec): [(0, 1), (1, 1), (5, 1)] ...
Término: 'broom'
  Aparece en 193 documentos.
  Posteos (ID_Doc, Frec): [(0, 1), (862, 1), (1344, 1)] ...


# 3. Modelos de Recuperación

### A. Modelo Binario Jaccard #

In [7]:
def busqueda_jaccard(consulta, df, indice_invertido):
    tokens_consulta = set(preprocesar_texto(consulta))
    
    if not tokens_consulta:
        return []
    
    # Buscar candidatos usando el índice invertido
    ids_candidatos = set()
    for token in tokens_consulta:
        if token in indice_invertido:
            for id_doc, _ in indice_invertido[token]:
                ids_candidatos.add(id_doc)
    
    resultados = []
    for id_doc in ids_candidatos:
        tokens_doc = set(df.at[id_doc, 'tokens'])
        
        interseccion = len(tokens_consulta.intersection(tokens_doc))
        union = len(tokens_consulta.union(tokens_doc))
        
        puntaje = interseccion / union if union > 0 else 0
        
        if puntaje > 0:
            resultados.append((id_doc, puntaje))
            
    return sorted(resultados, key=lambda x: x[1], reverse=True)

### TF-IDF

In [8]:
def busqueda_tfidf(consulta, df, diccionario_idf):
    tokens_consulta = preprocesar_texto(consulta)
    vector_consulta = generar_vector_tfidf(tokens_consulta, diccionario_idf)

    if not vector_consulta:
        return []
    
    resultados = []
    
    # Calcular similitud de coseno 
    for id_doc, vector_doc in enumerate(df['vector_tfidf']):
        puntaje = similitud_coseno(vector_consulta, vector_doc)
        
        if puntaje > 0:
            resultados.append((id_doc, puntaje))
            
    # Ordenar resultados 
    return sorted(resultados, key=lambda x: x[1], reverse=True)

### BM25

In [9]:
def calcular_longitud_promedio(df_tokens):
    total_len = sum(len(tokens) for tokens in df_tokens)
    return total_len / len(df_tokens)

LONGITUD_PROMEDIO = calcular_longitud_promedio(df['tokens'])
TOTAL_DOCS = len(df)

def busqueda_bm25(consulta, indice_invertido, df, k1=1.5, b=0.75):
    tokens_consulta = preprocesar_texto(consulta)
    puntajes = defaultdict(float)
    
    for token in tokens_consulta:
        if token not in indice_invertido:
            continue  
        # Recuperar lista de documentos que tienen el token
        lista_posteos = indice_invertido[token]
        docs_con_termino = len(lista_posteos)
        
        # Cálculo del IDF 
        numerador_idf = (TOTAL_DOCS - docs_con_termino + 0.5)
        denominador_idf = (docs_con_termino + 0.5)
        idf = math.log(numerador_idf / denominador_idf + 1)
        
        for id_doc, freq in lista_posteos:
            longitud_doc = len(df.at[id_doc, 'tokens'])
            
            numerador = freq * (k1 + 1)
            denominador = freq + k1 * (1 - b + b * (longitud_doc / LONGITUD_PROMEDIO))
            
            puntajes[id_doc] += idf * (numerador / denominador)
            
    return sorted(puntajes.items(), key=lambda x: x[1], reverse=True)

# 4. Funcion de cosulta y Ranking

In [10]:
def ejecutar_consulta(consulta, tipo_modelo='tfidf', top_n=10):
    print(f"\nConsulta: '{consulta}' | Modelo: {tipo_modelo.upper()}")
    print("-" * 60)
    
    if tipo_modelo == 'jaccard':
        resultados = busqueda_jaccard(consulta, df, indice_invertido)
        
    elif tipo_modelo == 'tfidf':
        resultados = busqueda_tfidf(consulta, df, diccionario_idf)
        
    elif tipo_modelo == 'bm25':
        resultados = busqueda_bm25(consulta, indice_invertido, df)
        
    else:
        print("Modelo no reconocido.")
        return

    # Mostrar top N resultados
    for rango, (id_doc, puntaje) in enumerate(resultados[:top_n], 1):
        titulo = df.at[id_doc, 'title']
        
        # Recortar descripción para visualización
        desc = df.at[id_doc, 'description'][:100] + "..." 
        
        print(f"Rango {rango} | Puntaje: {puntaje:.4f} | ID: {id_doc}")
        print(f"Vino: {titulo}")
        print(f"Desc: {desc}\n")


# Prueba con BM25
ejecutar_consulta("aromas of tropical fruit and citrus", tipo_modelo='bm25')

# Prueba con TF-IDF 
ejecutar_consulta("aromas of tropical fruit and citrus", tipo_modelo='tfidf')


Consulta: 'aromas of tropical fruit and citrus' | Modelo: BM25
------------------------------------------------------------
Rango 1 | Puntaje: 10.9850 | ID: 4033
Vino: Chateau Ste. Michelle 2015 Horse Heaven Vineyard Sauvignon Blanc (Horse Heaven Hills)
Desc: The aromas are light with notes of herb, citrus and tropical fruit. The palate is a bit more generou...

Rango 2 | Puntaje: 10.8324 | ID: 15976
Vino: Purple Star 2013 Sauvignon Blanc (Ancient Lakes)
Desc: Aromas of citrus, tropical fruit and herb lead to just off-dry flavors with a lightly creamy feel....

Rango 3 | Puntaje: 10.8324 | ID: 33477
Vino: Ancestry 2014 Réjouissant Sauvignon Blanc (Columbia Valley (WA))
Desc: Herb and citrus aromas lead to lightly spritzy, just off-dry tropical fruit flavors that linger....

Rango 4 | Puntaje: 10.5748 | ID: 23889
Vino: Washington Hills 2015 Sauvignon Blanc (Washington)
Desc: Light aromas of pineapple and other tropical fruit are accented by herb, floral and citrus flavors. ...

Rango 5

# 5. Evaluacion

In [11]:
def calcular_metricas(ids_recuperados, ids_relevantes):
    # Calcula Precision@K y Recall@K.
    if not ids_recuperados:
        return 0.0, 0.0
        
    relevantes_recuperados = [doc for doc in ids_recuperados if doc in ids_relevantes]
    
    precision = len(relevantes_recuperados) / len(ids_recuperados)
    recall = len(relevantes_recuperados) / len(ids_relevantes) if ids_relevantes else 0.0
    
    return precision, recall

def calcular_ap(ids_recuperados, ids_relevantes):
    
    if not ids_relevantes:
        return 0.0
        
    suma_precision = 0.0
    contador_relevantes = 0
    
    for i, id_doc in enumerate(ids_recuperados):
        if id_doc in ids_relevantes:
            contador_relevantes += 1
            precision_en_i = contador_relevantes / (i + 1)
            suma_precision += precision_en_i
            
    return suma_precision / len(ids_relevantes)

def evaluar_sistema(consultas_prueba, funcion_modelo):
    suma_map = 0
    
    print(f"{'Consulta':<30} | {'Precisión':<10} | {'Recall':<10} | {'AP':<10}")
    print("-" * 70)
    
    for consulta, ids_relevantes in consultas_prueba.items():
        # Ejecutar búsqueda con la función que pasemos
        resultados = funcion_modelo(consulta) 
        ids_recuperados = [id_doc for id_doc, _ in resultados]
        
        # Calcular métricas (Top 10 para P/R, lista completa para AP)
        p, r = calcular_metricas(ids_recuperados[:10], ids_relevantes)
        ap = calcular_ap(ids_recuperados, ids_relevantes)
        
        suma_map += ap
        print(f"{consulta[:28]:<30} | {p:.4f}      | {r:.4f}      | {ap:.4f}")
        
    print("-" * 70)
    print(f"Mean Average Precision (MAP): {suma_map / len(consultas_prueba):.4f}")

# 6. Ground truth

In [12]:
def generar_verdad_referencia(df, termino_consulta):
    
    indices_relevantes = set()
    # Usamos preprocesar para obtener la raíz (stem) y ser consistentes
    raiz_consulta = preprocesar_texto(termino_consulta)[0]
    
    for idx, tokens in enumerate(df['tokens']):
        if raiz_consulta in tokens:
            indices_relevantes.add(idx)
            
    return indices_relevantes

# Definimos consultas de prueba y generamos su referencia
consultas_prueba = {
    "tropical fruit": generar_verdad_referencia(df, "tropical"),
    "dry red": generar_verdad_referencia(df, "dry"),
    "sweet finish": generar_verdad_referencia(df, "sweet")
}

print(f"Referencias generadas para {len(consultas_prueba)} consultas de prueba.\n")

# Configuración de experimentos
modelos = ['jaccard', 'tfidf', 'bm25']
resultados_metricas = []

for nombre_modelo in modelos:
    print(f"Evaluando modelo: {nombre_modelo}...")
    
    suma_precision = 0
    suma_recall = 0
    puntaje_map = 0
    
    for texto_consulta, ids_relevantes in consultas_prueba.items():
        # Ejecutar búsqueda según el modelo
        if nombre_modelo == 'jaccard':
            resultados = busqueda_jaccard(texto_consulta, df, indice_invertido)
            
        elif nombre_modelo == 'tfidf':
            #  Usamos df y diccionario_idf 
            resultados = busqueda_tfidf(texto_consulta, df, diccionario_idf)
            
        elif nombre_modelo == 'bm25':
            resultados = busqueda_bm25(texto_consulta, indice_invertido, df)
            
        # Obtenemos solo los IDs recuperados
        ids_recuperados = [id_doc for id_doc, _ in resultados]
        
        # Calcular métricas individuales (Top 20 para P/R)
        p, r = calcular_metricas(ids_recuperados[:20], ids_relevantes)
        ap = calcular_ap(ids_recuperados, ids_relevantes)
        
        suma_precision += p
        suma_recall += r
        puntaje_map += ap
    
    # Promediar metricas entre todas las consultas para este modelo
    n_consultas = len(consultas_prueba)
    resultados_metricas.append({
        'Modelo': nombre_modelo.upper(),
        'Media Precision@20': round(suma_precision / n_consultas, 4),
        'Media Recall@20': round(suma_recall / n_consultas, 4),
        'MAP (Mean Avg Precision)': round(puntaje_map / n_consultas, 4)
    })

df_resultados = pd.DataFrame(resultados_metricas)
df_resultados

Referencias generadas para 3 consultas de prueba.

Evaluando modelo: jaccard...
Evaluando modelo: tfidf...
Evaluando modelo: bm25...


,Modelo,Media Precision@20,Media Recall@20,MAP (Mean Avg Precision)
0,JACCARD,0.9833,0.0025,0.6662
1,TFIDF,0.9667,0.0026,0.8437
2,BM25,1.0000,0.0026,0.8638


In [13]:
# Generar Verdad
consultas_reales = {
    "aromas of tropical fruit": generar_verdad_referencia(df, "tropical"),
    "dry red wine": generar_verdad_referencia(df, "dry")
}

print("\n    Modelo TF-IDF     ")
evaluar_sistema(consultas_reales, lambda q: busqueda_tfidf(q, df, diccionario_idf))

print("\n    Modelo BM25    ")
evaluar_sistema(consultas_reales, lambda q: busqueda_bm25(q, indice_invertido, df))


    Modelo TF-IDF     
Consulta                       | Precisión  | Recall     | AP        
----------------------------------------------------------------------
aromas of tropical fruit       | 1.0000      | 0.0027      | 1.0000
dry red wine                   | 1.0000      | 0.0004      | 0.5333
----------------------------------------------------------------------
Mean Average Precision (MAP): 0.7667

    Modelo BM25    
Consulta                       | Precisión  | Recall     | AP        
----------------------------------------------------------------------
aromas of tropical fruit       | 1.0000      | 0.0027      | 0.9998
dry red wine                   | 1.0000      | 0.0004      | 0.6146
----------------------------------------------------------------------
Mean Average Precision (MAP): 0.8072


# 7. Interfaz de CLI

In [14]:
## import sys

def iniciar_cli():
    print("      SISTEMA DE RECUPERACIÓN DE INFORMACIÓN DE VINOS")
    
    while True:
        print("\nOpciones:")
        print("1. Buscar")
        print("2. Salir")
        
        opcion = input("\nSeleccione una opcion: ").strip()
        
        if opcion == '2':
            print("Saliendo del sistema...")
            break
            
        if opcion == '1':
            consulta = input("Ingrese su consulta (ej. 'sweet tropical'): ").strip()
            if not consulta:
                print("La consulta no puede estar vacía.")
                continue
                
            print("\nModelos disponibles: [1] Jaccard, [2] TF-IDF, [3] BM25")
            opcion_modelo = input("Seleccione algoritmo (default BM25): ").strip()
            
            if opcion_modelo == '1':
                modelo = 'jaccard'
            elif opcion_modelo == '2':
                modelo = 'tfidf'
            else:
                modelo = 'bm25'
            
            try:
                ejecutar_consulta(consulta, tipo_modelo=modelo, top_n=5)
            except Exception as e:
                print(f"Error durante la búsqueda: {e}")
                import traceback
                traceback.print_exc()

        else:
            if opcion != '2':
                print("Opción no válida.")

# Ejecutar la interfaz
iniciar_cli()

      SISTEMA DE RECUPERACIÓN DE INFORMACIÓN DE VINOS

Opciones:
1. Buscar
2. Salir



Seleccione una opcion:  1
Ingrese su consulta (ej. 'sweet tropical'):  swett tropical



Modelos disponibles: [1] Jaccard, [2] TF-IDF, [3] BM25


Seleccione algoritmo (default BM25):  1



Consulta: 'swett tropical' | Modelo: JACCARD
------------------------------------------------------------
Rango 1 | Puntaje: 0.1111 | ID: 121733
Vino: McManis 2011 Viognier (California)
Desc: This is dry and crisp, with a trace of Viognier's tropical fruit and spice flavors....

Rango 2 | Puntaje: 0.1000 | ID: 108581
Vino: Matrix 2013 Matrix Estate Chardonnay (Russian River Valley)
Desc: This is a ripe, tropical and quite flabby wine, huge in body, oak and fruit....

Rango 3 | Puntaje: 0.1000 | ID: 52733
Vino: LangeTwins 2010 Estate Grown Chardonnay (Clarksburg)
Desc: Full of buttery, tropical flavors, it also has a barrel-infused taste of petroleum and char....

Rango 4 | Puntaje: 0.1000 | ID: 38468
Vino: Arrowood 2011 Chardonnay (Sonoma County)
Desc: This is a good, straightforward Chardonnay with buttercream, vanilla, tropical fruit and citrus flav...

Rango 5 | Puntaje: 0.1000 | ID: 96434
Vino: Ca' Momi 2011 Chardonnay (Napa Valley)
Desc: This Chardonnay has flavors of jammy tange

...


Modelos disponibles: [1] Jaccard, [2] TF-IDF, [3] BM25


Seleccione algoritmo (default BM25):  3



Consulta: 'sweet tropical' | Modelo: BM25
------------------------------------------------------------
Rango 1 | Puntaje: 8.5709 | ID: 116815
Vino: Santa Rita 2010 120 Chardonnay (Central Valley)
Desc: Melon, pear and apple aromas are standard and sweet. The palate is soft, maybe a bit mushy, with ban...

Rango 2 | Puntaje: 8.2785 | ID: 12069
Vino: Felten Cellars 2011 Sauvignon Blanc (Santa Barbara County)
Desc: This tastes sweet and simple, with lemongrass and tropical fruit notes. It's high in acidity, giving...

Rango 3 | Puntaje: 7.9080 | ID: 117240
Vino: Bota Box 2012 Chardonnay (California)
Desc: With sweet flavors of citrus and tropical fruits that are simple and syrupy, this Chardonnay is all ...

Rango 4 | Puntaje: 7.8575 | ID: 52306
Vino: Alta Vista 2011 Premium Torrontés (Salta)
Desc: Sweet, tropical aromas lead the way, followed by notes of melon and vanilla. It feels fresh, with ci...

Rango 5 | Puntaje: 7.7154 | ID: 4660
Vino: Château Bouscaut 2006 Barrel sample  (Pessac


Seleccione una opcion:  2


Saliendo del sistema...
